In [1]:
import numpy as np
import math as m
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.cross_validation import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

def calculate_mean_variance(X_train,Y_train):
    ind= range(len(Y_train))
    d={"x1":pd.Series(X_train[:,0], index=ind),"x2":pd.Series(X_train[:,1], index=ind), \
       "x3":pd.Series(X_train[:,2], index=ind),"x4":pd.Series(X_train[:,3], index=ind),"Label":pd.Series(Y_train)}
    df = pd.DataFrame(d)
    sf=pd.DataFrame(columns=('x1', 'x2', 'x3','x4'))
    vf=pd.DataFrame(columns=('x1', 'x2', 'x3','x4'))
    for i in range(len(Y_train)):
        if(df.Label[i] == 1):
              sf.loc[i]=[df.x1[i],df.x2[i],df.x3[i],df.x4[i]]
        else:
              vf.loc[i]=[df.x1[i],df.x2[i],df.x3[i],df.x4[i]]
    prior_setosa= float(len(sf))/float(len(df))
    prior_versi=float(len(vf))/float(len(df))
    cov_setosa=np.cov(sf.transpose())
    cov_versi=np.cov(vf.transpose())
    mean_setosa=[np.mean(sf.x1),np.mean(sf.x2),np.mean(sf.x3),np.mean(sf.x4)]
    mean_versi=[np.mean(vf.x1),np.mean(vf.x2),np.mean(vf.x3),np.mean(vf.x4)]
    return mean_setosa, mean_versi, cov_setosa, cov_versi, prior_setosa, prior_versi
    

def member(mean_setosa, mean_versi, cov_setosa, cov_versi,prior_setosa, prior_versi,X,Y):
    g1=[]
    g2=[]
    d=pd.DataFrame(columns=('d', 'Label'))
    for i in range(len(X)):
        g1x= -m.log(np.linalg.det(cov_setosa)) - (0.5 *(np.dot((X[i]-mean_setosa).transpose(),np.dot(np.linalg.inv(cov_setosa),\
                                                      (X[i]-mean_setosa))))) + m.log(prior_setosa)
        g1.append(g1x)
        g2x= -m.log(np.linalg.det(cov_versi)) - (0.5 *(np.dot((X[i]-mean_versi).transpose(),np.dot(np.linalg.inv(cov_versi),\
                                                      (X[i]-mean_versi))))) + m.log(prior_versi)
        g2.append(g2x)
#         discriminant= g1[i]-g2[i]
#         d.append(discriminant)
        d.loc[i]=[g1[i]-g2[i], Y[i]]
#     print d
    return d
    
def classify(d,Fun):
    c1=pd.DataFrame(columns=('c1','ALabel', 'PLabel','FunLabel'))
    c2=pd.DataFrame(columns=('c2', 'ALabel','PLabel','FunLabel'))
    for i in range(len(d)):
        if(d.d[i]>=0):
            c1.loc[i]=[d.d[i],d.Label[i],1,Fun[i]]
        else:
            c2.loc[i]=[d.d[i],d.Label[i],2,Fun[i]]
#     print c1
#     print c2
    c=pd.concat([c1,c2])
    return c
  
def confusion_mat(c):
    mse=mean_squared_error(c.ALabel, c.PLabel)
    print mse
    TP=0
    FP=0
    TN=0
    FN=0
    for i in range(len(c)):
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==1):
            TP+=1
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==2):
            TN+=1
        if(c.ALabel[i]==2 and c.PLabel[i]==1):
            FP+=1
        if(c.ALabel[i]==1 and c.PLabel[i]==2):
            FN+=1
    con_mat=np.matrix([[TP,FP],[FN,TN]])
    Accuracy=float((TP+TN))/float((TP+FP+TN+FN))
    print "Accuracy: ",Accuracy
    return con_mat
    
    
def k_fold(X, Y, k_fold):
    kf=sklearn.cross_validation.KFold(n=len(X), n_folds=10, shuffle=False,random_state=None)
    for train_index, test_index in kf:
        X_train,X_test=X[train_index], X[test_index]
        Y_train,Y_test=Y[train_index], Y[test_index]
        mean_setosa, mean_versi, cov_setosa, cov_versi,prior_setosa, prior_versi=calculate_mean_variance(X_train,Y_train)
        d=member(mean_setosa, mean_versi, cov_setosa, cov_versi,prior_setosa, prior_versi,X_train, Y_train)
        lda = LinearDiscriminantAnalysis()
        clf = lda.fit(X_train,Y_train)
        Fun1=clf.predict(X_train)
        c=classify(d,Fun1)
        con_mat=confusion_mat(c)
        p1= (float(con_mat[0,0])/float((con_mat[0,0]+con_mat[0,1])))
        r1=(float(con_mat[0,0])/float((con_mat[0,0]+con_mat[1,0])))
        p2= (float(con_mat[1,1])/float((con_mat[1,1]+con_mat[1,0])))
        r2=(float(con_mat[1,1])/float(con_mat[1,1]+con_mat[0,1]))
        precision=[p1,p2]
        recall=[r1,r2]
        print "Confusion Matrix: ",con_mat
        print "Precision: ",precision
        print "Recall: " ,recall
        F1=2*((p1*r1)/(p1+r1))
        F2=2*((p2*r2)/(p2+r2))
        print "F_measure: ",[F1,F2]
#         print X_train

def main():
    iris = load_iris()
    X = iris.data 
    Y = iris.target
    enc = LabelEncoder()
    label_encoder = enc.fit(Y)
    y = label_encoder.transform(Y) + 1
    x=X[0:100]
    y=y[0:100]
#     without fold
    print("Performance with 75% training size")
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
    mean_setosa, mean_versi, cov_setosa, cov_versi,prior_setosa, prior_versi=calculate_mean_variance(X_train,y_train)
    d=member(mean_setosa, mean_versi, cov_setosa, cov_versi,prior_setosa, prior_versi,X_test, y_test)
    lda = LinearDiscriminantAnalysis()
    clf = lda.fit(X_train,y_train)
    Fun=clf.predict(X_test)
    c=classify(d,Fun)
    con_mat=confusion_mat(c)
    p1= (float(con_mat[0,0])/float((con_mat[0,0]+con_mat[0,1])))
    r1=(float(con_mat[0,0])/float((con_mat[0,0]+con_mat[1,0])))
    p2= (float(con_mat[1,1])/float((con_mat[1,1]+con_mat[1,0])))
    r2=(float(con_mat[1,1])/float(con_mat[1,1]+con_mat[0,1]))
    precision=[p1,p2]
    recall=[r1,r2]
    print "Confusion Matrix: ",con_mat
    print "Precision: ",precision
    print "Recall: " ,recall
    F1=2*((p1*r1)/(p1+r1))
    F2=2*((p2*r2)/(p2+r2))
    print "F_measure: ",[F1,F2]
    p=dict()
    r=dict()
    av_p=dict()
    for i in range(2):
        p[i], r[i], _ = precision_recall_curve(c.ALabel,
                                                c.PLabel,pos_label=1)
#         av_p[i] = average_precision_score(c.ALabel, c.PLabel)

    for i in range(2):
        plt.plot(r[i], p[i])
#     for i in range(2):
#         plt.plot(precision[i],recall[i])
    plt.xlim([0.0, 1.5])
    plt.ylim([0.0, 1.5])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall curve')
    plt.show()

#     With fold
    print("******")
    print("Performance with 10 fold")
    k_fold(x,y,10)
    
    
    
if __name__ == "__main__":main()

Performance with 75% training size
0.0
Accuracy:  1.0
Confusion Matrix:  [[14  0]
 [ 0 11]]
Precision:  [1.0, 1.0]
Recall:  [1.0, 1.0]
F_measure:  [1.0, 1.0]
******
Performance with 10 fold
0.0
Accuracy:  1.0
Confusion Matrix:  [[40  0]
 [ 0 50]]
Precision:  [1.0, 1.0]
Recall:  [1.0, 1.0]
F_measure:  [1.0, 1.0]
0.0
Accuracy:  1.0
Confusion Matrix:  [[40  0]
 [ 0 50]]
Precision:  [1.0, 1.0]
Recall:  [1.0, 1.0]
F_measure:  [1.0, 1.0]
0.0
Accuracy:  1.0
Confusion Matrix:  [[40  0]
 [ 0 50]]
Precision:  [1.0, 1.0]
Recall:  [1.0, 1.0]
F_measure:  [1.0, 1.0]
0.0
Accuracy:  1.0
Confusion Matrix:  [[40  0]
 [ 0 50]]
Precision:  [1.0, 1.0]
Recall:  [1.0, 1.0]
F_measure:  [1.0, 1.0]
0.0
Accuracy:  1.0
Confusion Matrix:  [[40  0]
 [ 0 50]]
Precision:  [1.0, 1.0]
Recall:  [1.0, 1.0]
F_measure:  [1.0, 1.0]
0.0
Accuracy:  1.0
Confusion Matrix:  [[50  0]
 [ 0 40]]
Precision:  [1.0, 1.0]
Recall:  [1.0, 1.0]
F_measure:  [1.0, 1.0]
0.0
Accuracy:  1.0
Confusion Matrix:  [[50  0]
 [ 0 40]]
Precision:  [1.